In [13]:
using Distributed
using Base.Threads
@everywhere begin
    using BenchmarkTools
    using ParallelTemperingMonteCarlo
end
#addprocs(nthreads())

8-element Vector{Int64}:
 18
 19
 20
 21
 22
 23
 24
 25

In order to use a function of several variables in the pmap environment we require a curry function

In [6]:
@everywhere curry(f,y) = x -> f(x,y)
@everywhere add_xy(x,y) = x + y 


In [12]:
pmap(curry(add_xy,10), 1:5)


5-element Vector{Int64}:
 11
 12
 13
 14
 15

NB spawning processes is expensive and _not recommended at all_ for simple loops and functions.

# Sync macros

sync requires all tasks inside to complete before moving on, async moves right on along without waiting, 

In [20]:
@sync begin
    sleep(2)
    println("slept for two")

    @async begin 
        sleep(5)
        println("nice and rested")
    end
#the async wrapper skips straight to done
    println("done")
end

slept for two
done


nice and rested


We'll simulate a complex process with a 2 second sleep

In [22]:
function simtest(x)
    sleep(2)
    return x
end

simtest (generic function with 1 method)

In [30]:
@time begin 
    veccy = []
    for i = 1:10

        y = simtest(i)
        push!(veccy,y)
        
    end
    println(veccy)
end

@time begin 
    @sync for i = 1:10
        veccy = []
        @async begin 
            y = simtest(i)
            push!(veccy,y)
        end
    end
    println(veccy)
end

Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
 20.016610 seconds (1.16 k allocations: 35.969 KiB)


Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  2.024840 seconds (2.64 k allocations: 133.564 KiB, 1.09% compilation time)


NB: Asynchronous tasks are not parallel, demonstrated above they do boost performance for large operations. Use @spawn not @async for parallel operations

Let's test a data unsafe operation

In [42]:
function printtest(x::Bool = true)
    println("begin")
    if x == true
        @sync for i=1:10
            @spawn println("$i $i $i $i $i ")
        end
    else
        for i=1:10
            println("$i $i $i $i $i ")
        end
    end
    println("end")
end

printtest (generic function with 2 methods)

In [44]:
@time printtest()
println("done")
@time printtest(false)
println("done")
@time begin 
    println("begin")
    @threads for i=1:10
        println("$i $i $i $i $i ")
    end
    println("end")
end

begin
      From worker 3:	6 6 6 6 6 
      From worker 4:	7 7 7 7 7 
      From worker 2:	5 5 5 5 5 
      From worker 5:	8 8 8 8 8 
      From worker 7:	10 10 10 10 10 
      From worker 6:	9 9 9 9 9 


      From worker 22:	1 1 1 1 1 
      From worker 24:	3 3 3 3 3 
      From worker 23:	2 2 2 2 2 
      From worker 25:	4 4 4 4 4 


end
  3.457296 seconds (2.62 k allocations: 123.656 KiB)
done
begin
1 1 1 1 1 
2 2 2 2 2 
3 3 3 3 3 
4 4 4 4 4 
5 5 5 5 5 
6 6 6 6 6 
7 7 7 7 7 
8 8 8 8 8 
9 9 9 9 9 
10 10 10 10 10 
end
  0.000178 seconds (517 allocations: 17.969 KiB)
done
begin
7 7 7 7 7 
1 1 1 1 1 
2 2 2 2 2 
10 10 10 10 10 
3 3 3 3 3 
4 4 4 4 4 
5 5 5 5 5 
8 8 8 8 8 
6 6 6 6 6 
9 9 9 9 9 
end
  0.023980 seconds (18.15 k allocations: 1.024 MiB, 96.25% compilation time)


# NB 
Categorically do not use @spawn where data ordering is relevant. Additionally, it is considered bad practice to parallelise any operation faster than 100 $\mu m$ for threads or 100ms for spawning as it does not increase speed enough to compensate for the time taken to spawn the operation